# LightGBM (Gradient Boosting Machine) Tutorial
- Kuzushi-ji train
- [URL](https://www.codexa.net/lightgbm-beginner/)

## Upload file

In [2]:
from google.colab import files
uploaded = files.upload()

Saving k49_classmap.csv to k49_classmap.csv
Saving k49-train-imgs.npz to k49-train-imgs.npz
Saving k49-train-labels.npz to k49-train-labels.npz
Saving k49-test-imgs.npz to k49-test-imgs.npz
Saving k49-test-labels.npz to k49-test-labels.npz


## Import library

In [3]:
# データ処理ラブラり
import numpy as np
import pandas as pd
# データ可視化ライブラリ
import matplotlib.pyplot as plt
%matplotlib inline
# LightGBM
import lightgbm as lgb
# Scikit-learn（評価算出）
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

## Check data

In [4]:
classes = pd.read_csv('kmnist_classmap.csv')
print(classes.shape)
classes

FileNotFoundError: [Errno 2] No such file or directory: 'kmnist_classmap.csv'

## Arrange data

In [ ]:
# 訓練データの読み込み
X_train = np.load('kmnist-train-imgs.npz')['arr_0']
y_train = np.load('kmnist-train-labels.npz')['arr_0']

# テストデータの読み込み
X_test = np.load('kmnist-test-imgs.npz')['arr_0']
y_test = np.load('kmnist-test-labels.npz')['arr_0']

# データのサイズ確認
print(X_train.shape)
print(X_test.shape)


print(y_train[0:5])
print(classes)

print(y_train[0:5])
print(classes)


labelindex = classes.set_index('index').to_dict()['char']
labelindex

print(X_train.min())
print(X_train.max())

plt.imshow(X_train[423], cmap = plt.cm.gray)
plt.show()


print(y_train[423])
print(labelindex[1])

print(y_train[423])
print(labelindex[1])


## Preprocess data

In [ ]:
#normalization
X_train = X_train / 255
X_test = X_test/ 255

#reshape data from 28*28 to 784
X_train = X_train.reshape(X_train.shape[0], 784)
X_test = X_test.reshape(X_test.shape[0], 784)

print(X_train.shape)
print(X_test.shape)


## Model definition

In [ ]:
lgb.LGBMClassifier()

## Train
- num_leaves \
LightGBMで最も重要と言っても過言ではないのがnum_leaves(葉の数)です。num_leavesは決定木の複雑度を調整します。num_leavesの値が高すぎると過学習となり、低すぎると未学習になります。num_leavesを調整する場合はmax_depth（決定木の深さ）のパラメータと一緒に調整すると良いです。

- min_data_in_leaf \
Leaf-wiseを採用しているLightGBMではmin_data_in_leafはとても重要なハイパーパラメータです。min_data_in_leafは決定木のノード（葉）の最小データ数を指定します。値が高いと決定木が深く育つのを抑えるため過学習防ぎますが、逆に未学習となる場合もあります。min_data_in_leafは訓練データのレコード数とnum_leavesに大きく影響されます。

- max_depth \
決定木の深さを指定するハイパーパラメータです。単体で調整するよりも、他のハイパーパラメータとのバランスを考えながら調整します。

### Tips
- モデル訓練のスピードをあげる\
bagging_fraction（初期値1.0）とbagging_freq（初期値0）を使う\
feature_fraction（初期値1.0）で特徴量のサブサンプリングを指定\
小さいmax_bin（初期値 255）を使う\
save_binary（初期値 False）を使う\
分散学習を使う[公式ガイドはこちら](https://lightgbm.readthedocs.io/en/latest/Parallel-Learning-Guide.html)\
- 推測精度を向上させる\
大きいmax_bin（初期値255）を使う\
小さいlearning_rate(初期値0.1)と大きいnum_iterations(初期値100)を使う\
大きいnum_leaves（初期値31）を使う\
訓練データのレコード数を増やす（可能であれば）\
- 過学習対策\
小さいmax_binを使う（初期値255）\
小さいnum_leavesを使う（初期値31）\
min_data_in_leaf（初期値20）とmin_sum_hessian_in_leaf(初期値1e-3)を使う\
bagging_fraction（初期値1.0）とbagging_freq（初期値0）を使う\
feature_fraction（初期値1.0）で特徴量のサブサンプリングを指定\
訓練データのレコード数を増やす（可能であれば）\
lambda_l1（初期値0.0）、lambda_l2（初期値0.0）、min_gain_to_split（初期値0.0）で正則化を試す\
max_depth（初期値-1）を指定して決定木が深くならないよう調整する\

In [ ]:
train_data = lgb.Dataset(X_train, label=y_train)
eval_data = lgb.Dataset(X_test, label=y_test, reference= train_data)

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 10,
    'verbose': 2,
}



In [ ]:

gbm = lgb.train(
params,
train_data,
valid_sets=eval_data,
num_boost_round=100,
verbose_eval=5,
)

## Prediction

In [ ]:
preds = gbm.predict(X_test)

y_pred = []
for x in preds:
  y_pred.append(np.argmax(x))

confusion_matrix(y_test, y_pred)

accuracy_score(y_test, y_pred)